In [ ]:
# import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pandas as pd
import pymongo
import time
import datetime as dt

In [ ]:
# Initialize browser
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# URL of page to be scraped - NASA Mars News Site 
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
time.sleep(5)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup_nasa = BeautifulSoup(html, 'html.parser')

In [ ]:
# Collect the latest News Title from NASA Mars News Site
news_info = []
  
news_title = soup_nasa.find('div', class_='content_title').find('a').text
news_title

In [ ]:
# Latest news paragraph text from NASA Mars News Site
news_p = soup_nasa.find('div', class_='article_teaser_body').text
news_p

In [ ]:
# Append the retreived information into a list of dictionaries 
news_info.append({"news_title" : news_title, "news Paragraph" : news_p})
news_info

In [ ]:
# URL of page to be scraped for featured image - JPL Mars Space Images
jpl_url_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url_image)
time.sleep(5)

In [ ]:
# Browse through the page
# Find the image url for the current Featured Mars and click the full image button
featured_full_image = browser.find_by_id('full_image')
featured_full_image.click()

In [ ]:
# Browse through the page
time.sleep(5)

# Find the more info button and click
more_info_images = browser.find_link_by_partial_text('more info')
more_info_images.click()

In [ ]:
# Using BeautifulSoup create an object and parse with 'html.parser'
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# find the related image url
image_url = image_soup.find('figure', class_='lede').find('img')['src']
image_url

In [ ]:
# Use the base url to create an full url
JPL_link = 'https://www.jpl.nasa.gov'
featured_image_url = JPL_link + image_url
featured_image_url

In [ ]:
# URL of page to be scraped - Mars Weather
mars_twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_twitter_url)
time.sleep(5)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
html_weather = browser.html

mars_soup = BeautifulSoup(html_weather, 'html.parser')

In [ ]:
# Find all elements that contain tweets
latest_tweets = mars_soup.find_all('div', class_='js-tweet-text-container')
latest_tweets

In [ ]:

# Loop through latest_tweets to extract for the weather report

for tweet in latest_tweets: 
    
    weather_tweet = tweet.find('p').text
    
    if 'Sol' and 'pressure' in weather_tweet:
        
        print(weather_tweet)
        
        break
        
    else: 
        
        pass

In [ ]:
latest_weather_tweet = weather_tweet.split("hPa")[0]+"hPa"
print(latest_weather_tweet)

In [ ]:
# URL of page to be scraped - Mars Facts
mars_facts_url = 'http://space-facts.com/mars/'
browser.visit(mars_facts_url)
time.sleep(5)

In [ ]:
# Use Panda's `read_html` to parse the url
#mars_facts = pd.read_html(mars_facts_url)
mars_facts = pd.read_html(browser.html)
mars_facts

In [ ]:
# Find the mars facts DataFrame in the list of DataFrames 
mars_df = mars_facts[1]
mars_df

In [ ]:
# Set columns to ['Description', 'Value']
mars_df.columns = ['Description', 'Value']
mars_df

In [ ]:
# Set the index to the `Description` column without row indexing
mars_df.set_index('Description', inplace=True)

In [ ]:
# Use pandas to generate Html Tables from dataframes and save as html file
mars_fact_table = mars_df.to_html('mars_fact_table.html')
mars_fact_table

In [ ]:
# URL of page to be scraped - Mars Hemispheres
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
time.sleep(5)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
hemispheres_soup = BeautifulSoup(html_hemispheres, 'html.parser')

In [ ]:
# Retreive all items that contain mars hemispheres information
items = hemispheres_soup.find_all('div', class_='item')

# Create an empty list to hold dictionaries of hemisphere title with the image url string
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

In [ ]:
# Loop through the items previously stored
for item in items: 
    # Store title
    title = item.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = item.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    individual_soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + individual_soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemi_urls = {"title" : title, "img_url" : img_url}
    #hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    
    # Append the retreived information 
    hemisphere_image_urls.append(hemi_urls)

# Display hemisphere_image_urls
hemisphere_image_urls